In [305]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime

In [306]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')

    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass

    for linebreak in soup.find_all('br'):
        linebreak.extract()

    return soup

In [307]:
def remove_multiple_spaces(string):
    pattern = r'\s+'
    replaced_string = re.sub(pattern, ' ', string)
    return replaced_string


def find_qrt_date(content):
    qtr_date = content.find_all(text=re.compile(
        r'for\s+(the\s+)?(fiscal\s+)?year\s+ended\s+|for\s+the\s+quarter\s+ended\s+|for\s+the\s+quarterly\s+period\s+ended\s+', re.IGNORECASE))
    qtr_match = re.search(
        r'([A-Za-z]+)\s+(\d{1,2}),\s+(\d{4})', qtr_date[0].replace('\n', ''))
    if qtr_match is None:
        qtr_match = qtr_match = re.search(
            r'([A-Za-z]+) (\d{1,2}), (\d{4})', qtr_date[1])
    return remove_multiple_spaces(str(qtr_match.group()))

In [308]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
filing_links = pd.read_excel(
    "/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC__sec_filing_links.xlsx")
filing_links.head()

,Form type,Form description,Filing date,Reporting date,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],2023-07-25,2023-06-30,https://www.sec.gov/Archives/edgar/data/128775...
1,10-Q,Quarterly report [Sections 13 or 15(d)],2023-04-25,2023-03-31,https://www.sec.gov/Archives/edgar/data/128775...
2,10-K/A,"Annual report [Section 13 and 15(d), not S-K I...",2023-03-31,2022-12-31,https://www.sec.gov/Archives/edgar/data/128775...
3,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2023-02-07,2022-12-31,https://www.sec.gov/Archives/edgar/data/128775...
4,10-Q,Quarterly report [Sections 13 or 15(d)],2022-10-25,2022-09-30,https://www.sec.gov/Archives/edgar/data/128775...


In [309]:
# drops all the amendment filing
filing_links = filing_links.drop(filing_links[filing_links['Form description'].str.contains(
    'amendment', case=False)].index).reset_index(drop=True)
filing_links.head()

,Form type,Form description,Filing date,Reporting date,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],2023-07-25,2023-06-30,https://www.sec.gov/Archives/edgar/data/128775...
1,10-Q,Quarterly report [Sections 13 or 15(d)],2023-04-25,2023-03-31,https://www.sec.gov/Archives/edgar/data/128775...
2,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2023-02-07,2022-12-31,https://www.sec.gov/Archives/edgar/data/128775...
3,10-Q,Quarterly report [Sections 13 or 15(d)],2022-10-25,2022-09-30,https://www.sec.gov/Archives/edgar/data/128775...
4,10-Q,Quarterly report [Sections 13 or 15(d)],2022-07-26,2022-06-30,https://www.sec.gov/Archives/edgar/data/128775...


In [310]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")

In [311]:
filing_links.head()

,Form type,Form description,Filing date,Reporting date,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],"July 25, 2023","June 30, 2023",https://www.sec.gov/Archives/edgar/data/128775...
1,10-Q,Quarterly report [Sections 13 or 15(d)],"April 25, 2023","March 31, 2023",https://www.sec.gov/Archives/edgar/data/128775...
2,10-K,"Annual report [Section 13 and 15(d), not S-K I...","February 07, 2023","December 31, 2022",https://www.sec.gov/Archives/edgar/data/128775...
3,10-Q,Quarterly report [Sections 13 or 15(d)],"October 25, 2022","September 30, 2022",https://www.sec.gov/Archives/edgar/data/128775...
4,10-Q,Quarterly report [Sections 13 or 15(d)],"July 26, 2022","June 30, 2022",https://www.sec.gov/Archives/edgar/data/128775...


In [312]:
filing_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Form type         52 non-null     object
 1   Form description  52 non-null     object
 2   Filing date       52 non-null     object
 3   Reporting date    52 non-null     object
 4   Filings URL       52 non-null     object
dtypes: object(5)
memory usage: 2.2+ KB


In [313]:
url = filing_links.iloc[0]['Filings URL']
date = filing_links.iloc[0]['Reporting date']
url, date

('https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm',
 'June 30, 2023')

In [314]:
response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')

In [315]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    all_tags = soup_content.find_all(True)
    print(type(all_tags))
    count = 0
    for tag in soup_content.find_all(text=re.compile('^.*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS.*$')):
        date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', tag)
        if date_str is None:
            next_line = tag.find_next(text=re.compile(
                r'([A-Za-z]+) (\d{1,2}), (\d{4})')).text
            date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', next_line)
        if date_str is None:
            next_line = tag.next.next.next.next.next.next.text
            date_str = re.search(r'([A-Za-z]+) (\d{1,2}), (\d{4})', next_line)
        if date_str is not None:
            date_str = str(date_str.group())
            date_str = unicodedata.normalize('NFKD', date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                print('Table found: ')
                print('Table #', count)
                html_table = tag.find_next('table')
                if master_table is None:
                    master_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    master_table = master_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                              regex=True)
                    master_table = master_table.dropna(how='all', axis=0)
                else:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan,
                                                                                        regex=True)
                    new_table = new_table.dropna(how='all', axis=0)
                    # print('head')
                    # print(new_table.head()) # text
                    master_table = master_table.append(
                        new_table.dropna(how='all', axis=0).reset_index(
                            drop=True).drop(index=0),
                        ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table

In [316]:
master_table = extract_tables(content, date)

<class 'bs4.element.ResultSet'>
Table found: 
Table # 1
Table found: 
Table # 2


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/1603313143.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/1603313143.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


Table found: 
Table # 3
Table found: 
Table # 4


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/1603313143.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/1603313143.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


Table found: 
Table # 5
Table found: 
Table # 6


...

Table found: 
Table # 50
Table found: 
Table # 51


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/1603313143.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/1603313143.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


Table found: 
Table # 52
Table found: 
Table # 53


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/1603313143.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(
/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/1603313143.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_table = master_table.append(


Table found: 
Table # 54


In [317]:
master_table

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,Company (1),NaN,Business Description,NaN,Investment,NaN,Coupon (3),Reference (7),Spread (3),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Software and Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"2U, Inc.",NaN,Provider of course design and learning managem...,NaN,First lien senior secured loan,NaN,11.32 %,SOFR (M),6.50 %,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"AffiniPay Midco, LLC and AffiniPay Intermediat...",NaN,Payment processing solution provider,NaN,First lien senior secured loan,NaN,10.20 %,SOFR (A),5.50 %,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,First lien senior secured loan,NaN,10.39 %,SOFR (A),5.50 %,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,NaN,NaN,NaN,NaN,First lien senior secured loan,NaN,11.00 %,SOFR (Q),5.50 %,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1504,NaN,NaN,NaN,NaN,Series A preferred stock,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
master_table.to_csv('8_master_table.csv')

In [319]:
def process_table(soi_table_df, append_str):
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    soi_table_df = soi_table_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    print('4: ' + str(soi_table_df.shape))

    # Separate header and data
    soi_table_header = soi_table_df.iloc[0].dropna(how='any')
    print('header: ')
    print(soi_table_header)
    soi_table_data_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    print('1: ' + str(soi_table_data_df.shape))
    print('4: ' + str(soi_table_data_df.shape))

    # Drop rows with only two non-null values
    # soi_table_data_df = soi_table_data_df.dropna(thresh=3)
    # print('4: ' + str(soi_table_data_df.shape))

    columns_to_fill = ['Amortized Cost', 'Fair Value', '% of Net Assets']
    for col in columns_to_fill:
        col_index = soi_table_data_df.columns.get_loc(col)
        next_col_index = col_index + 1

        for i in range(len(soi_table_data_df)):
            current_value = soi_table_data_df.iat[i, col_index]

            if pd.isna(current_value) and next_col_index < len(soi_table_data_df.columns):
                next_valid_index = next((j for j, v in enumerate(
                    soi_table_data_df.iloc[i, next_col_index:], start=next_col_index) if pd.notna(v)), None)

                if next_valid_index is not None:
                    next_value = soi_table_data_df.iat[i, next_valid_index]
                    soi_table_data_df.iat[i, col_index] = next_value
                    soi_table_data_df.iat[i, next_valid_index] = pd.NA

    # Drop rows labeled as subtotals
    subtotal_rows = soi_table_data_df[soi_table_data_df['Company (1)'].str.contains(
        'subtotal', case=False, na=False)]
    soi_table_data_df = soi_table_data_df[~soi_table_data_df.index.isin(
        subtotal_rows.index)]
    print('2: ' + str(soi_table_data_df.shape))

    # Drop rows based on regex pattern (e.g., 'subtotal' or 'total')
    pattern = r'^([Ss]ubtotal)|([Tt]otal)'
    mask = soi_table_data_df.apply(lambda row: row.astype(
        str).str.contains(pattern, case=False, na=False)).any(axis=1)
    soi_table_data_df = soi_table_data_df[~mask]
    print('2: ' + str(soi_table_data_df.shape))

    # Drop rows with all missing values
    soi_table_df = soi_table_df.dropna(how='all')
    print('4: ' + str(soi_table_data_df.shape))

    # # Drop columns with all missing values
    soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    print('4: ' + str(soi_table_data_df.shape))

    # Forward fill the first two columns
    col_indices = [0, 1]
    soi_table_data_df.iloc[:, col_indices] = soi_table_data_df.iloc[:, col_indices].fillna(
        method='ffill')

    soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)

    num_cols = soi_table_data_df.shape[1]
    for col_index in range(num_cols-3, num_cols):
        col_name = soi_table_data_df.columns[col_index]
        soi_table_data_df[col_name] = pd.to_numeric(
            soi_table_data_df[col_name], errors='coerce')

    return soi_table_data_df


process_table_ = process_table(master_table, "hi")
process_table_.to_excel("ex.xlsx")
process_table_.to_csv('ex.csv')
process_table_

4: (1508, 21)
header: 
0              Company (1)
2     Business Description
4               Investment
6               Coupon (3)
7            Reference (7)
8               Spread (3)
10        Acquisition Date
12           Maturity Date
14            Shares/Units
15               Principal
17          Amortized Cost
19              Fair Value
21         % of Net Assets
Name: 0, dtype: object
1: (1507, 21)
4: (1507, 21)
2: (1507, 21)
2: (1506, 21)
4: (1506, 21)
4: (1506, 14)


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_14262/2085212174.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = soi_table_data_df.apply(lambda row: row.astype(


,Company (1),Business Description,Investment,Coupon (3),Reference (7),Spread (3),Acquisition Date,Maturity Date,Shares/Units,Principal,NaN,Amortized Cost,Fair Value,% of Net Assets
1,Software and Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"2U, Inc.",Provider of course design and learning managem...,First lien senior secured loan,11.32 %,SOFR (M),6.50 %,01/2023,12/2026,NaN,NaN,4.7,4.4,4.5,NaN
3,"AffiniPay Midco, LLC and AffiniPay Intermediat...",Payment processing solution provider,First lien senior secured loan,10.20 %,SOFR (A),5.50 %,02/2020,06/2028,NaN,63.0,NaN,63.0,61.8,NaN
4,"AffiniPay Midco, LLC and AffiniPay Intermediat...",Payment processing solution provider,First lien senior secured loan,10.39 %,SOFR (A),5.50 %,06/2022,06/2028,NaN,120.0,NaN,118.0,117.6,NaN
5,"AffiniPay Midco, LLC and AffiniPay Intermediat...",Payment processing solution provider,Senior subordinated loan,15.06 % PIK,SOFR (Q),10.00 %,02/2020,06/2030,NaN,61.0,NaN,61.0,59.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,"Flinn Scientific, Inc. and WCI-Quantum Holding...","Distributor of instructional products, service...",First lien senior secured loan,11.00 %,SOFR (Q),5.50 %,07/2017,08/2024,NaN,29.4,NaN,29.4,29.4,NaN
1503,"Flinn Scientific, Inc. and WCI-Quantum Holding...","Distributor of instructional products, service...",First lien senior secured loan,11.00 %,SOFR (Q),5.50 %,08/2018,08/2024,NaN,1.1,NaN,1.1,1.1,NaN
1504,"Flinn Scientific, Inc. and WCI-Quantum Holding...","Distributor of instructional products, service...",Series A preferred stock,NaN,NaN,NaN,10/2014,NaN,1272,NaN,NaN,0.7,1.4,NaN
1505,"Flinn Scientific, Inc. and WCI-Quantum Holding...","Distributor of instructional products, service...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.5,42.2,NaN
